# Modifications de résultats des sondages

## Import préliminaires des librairies

In [57]:
import re
import numpy as np
import pandas as pd
import locale 
from locale import atof
# Régression linéaire
from sklearn.linear_model import LinearRegression

# Read files
import PyPDF2
from pdfminer.high_level import extract_text

# Perso
from importlib.machinery import SourceFileLoader
tools = SourceFileLoader('methods', '../methods.py').load_module()
from methods import *

In [58]:
fp="rapport/2020-05-05.pdf"
text = extract_text(fp,page_numbers=[16])
question1="Êtes-vous  satisfait  ou  mécontent  d"
question2="Êtes-vous satisfait ou mécontent d"
# print(text.find(question1))
# print(text.find(question2))
# print(len(question2))
print(re.sub('\.\.\. (?![\n])','...\n',text))

Le classement des personnalités
Auprès des sympathisants de la gauche

Question : Pour chacune des personnalités suivantes, indiquez si vous en avez une image positive ou une image négative ?
En %

51

16

9

7

Nicolas Hulot
Olivier Besancenot
Jean-Luc Mélenchon
Christiane Taubira
Anne Hidalgo
Benoît Hamon
François Hollande
Ségolène Royal
Yannick Jadot
François Ruffin
Arnaud Montebourg
Bernard Cazeneuve
Olivier Véran
Jean-Yves Le Drian
Olivier Faure
François Bayrou
Xavier Bertrand
Bruno Le Maire
François Baroin
Valérie Pécresse
Jean-Michel Blanquer
Nicolas Sarkozy
Manuel Valls
Rachida Dati
Marlène Schiappa
Christophe Castaner
Marion Maréchal
Nicolas Dupont-Aignan
Christian Jacob
Sibeth Ndiaye
Marine Le Pen
Gérald Darmanin
Gérard Larcher
Muriel Pénicaud
Laurent Wauquiez

8

13

16

8
10
6
5

9
9

4
4
3
4
3
3
2
2
1
2
2
2
2
2
1
1
1
3

1
1
1
1

18
16
16
16
15
13
14
14
13
10

12
11
11
10
10
10
7

44
36
32

38
35

31

32

26
26

30
28
28
25

23
23

22
22

30

21

18

25
25
27

23

28

31

3

In [59]:
with open(fp,"rb") as pdf_file:
    pdf_reader= PyPDF2.PdfFileReader(pdf_file)
    isEncrypted = pdf_reader.getIsEncrypted()
    nbPages= pdf_reader.numPages

    # Define columns
    page_content=pdf_reader.getPage(15).extractText()
print(page_content)

10
6
6
5
6
5
6
5
8
3
4
6
6
3
2
2
3
2
1
3
3
2
2
3
2
2
4
3
2
2
2
1
1
1
1
38
28
28
25
23
24
22
22
18
22
20
17
17
20
21
21
19
20
20
17
17
18
18
16
17
17
14
14
14
14
13
12
11
11
7
19
18
23
19
18
21
17
17
17
26
23
21
18
26
23
24
25
28
25
28
21
27
25
17
20
30
17
23
23
22
18
28
20
22
18
15
18
28
19
18
22
18
29
45
27
39
36
39
29
23
24
23
24
35
35
36
20
28
19
25
33
24
27
22
33
20
40
22
31
47
18
30
15
32
35
28
37
27
12
22
14
20
20
22
31
29
30
26
19
17
23
33
27
45
36
18
41
33
39
29
47
19
46
35
27
Nicolas Hulot
Olivier Véran
Nicolas Sarkozy
Xavier Bertrand
Jean-Yves Le Drian
Bruno Le Maire
François Baroin
Olivier Besancenot
Marine Le Pen
Anne Hidalgo
Jean-Luc Mélenchon
Christiane Taubira
Marion Maréchal
Rachida Dati
Bernard Cazeneuve
Valérie Pécresse
Jean-Michel Blanquer
François Bayrou
Christophe Castaner
François Hollande
Nicolas Dupont-Aignan
Arnaud Montebourg
Benoît Hamon
Yannick Jadot
Gérald Darmanin
Ségolène Royal
François Ruffin
Laurent Wauquiez
Gérard Larcher
Marlène Schiappa
Christian Jaco

# Extraction des données avec PyPDF2
PyPDF2 semble être constant dans la forme d'extraction des données de Elab.
La structure des données ainsi que les détails d'extraction sont précisés ci-dessous:
* Elabe fourni les données détaillées pour deux personnalités que sont le président de la République et le premier ministre. Les noms n'étant pas indiqués sur la page de données, il faut les extraire ailleurs ou les préciser à la main.
* Il y a deux pages successives de données par individus
* Les catégories principales sont indiquées en capitales
* Il existe des catégories secondaires et tertiaires sans différenciation possible. Elles seront traités comme sous catégories.
* La base de chaque catégories n'est pas précisée, il faudra donc utiliser de la régression linéaire pour tenter de les retrouver
* Les trous dans les données ne sont pas visible. Il faut un système de vérification.
* Les variations ne sont pas données

__Format des données__
* Les données sont fournis en lisant les lignes dans l'ordre
* Chaque cellule est extraite par PyPDF2 une à la fois en introduisant un saut de ligne
* Les blancs dans les données ne créént pas de fusion d'information mais disparaissent sans faire de double saut de lignes
* Le caractère '__-__' compromet les cellules et induit un ou plusieurs sauts malencontreux :
    * Si le '-' apparait au milieu d'un texte, il induit un _double saut de ligne_ (ex: 18-24 ans)
    * Si le '-' apparait à la fin d'un texte il induit un _seul_ saut de ligne supplémentaire avant sont apparition (CSP-)  
* Il faudra donc préformater le texte au préalable


## Extract content (to delete later)

In [60]:
def get_page_content(num_page,fp)->str:
    with open(fp,"rb") as pdf_file:
        pdf_reader= PyPDF2.PdfFileReader(pdf_file)
        isEncrypted = pdf_reader.getIsEncrypted()
        nbPages= pdf_reader.numPages
        return pdf_reader.getPage(num_page-1).extractText()

    # Define columns
page_content=get_page_content(2,fp)
page_content

"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de \n1 009  \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n4 \net \n5 mai 2020\n.\n07/05/2020\nL'Observatoire politique \n\nMai 2020\n"

## Préformatage

In [61]:
##Préformatage
def preformatage(page_content)->str:
    #Remplacement des données d'age
    page_content=page_content.replace('18\n-\n24 ans','18-24 ans')
    page_content=page_content.replace('25\n-\n34 ans','25-34 ans')
    page_content=page_content.replace('35\n-\n49 ans','35-49 ans')
    page_content=page_content.replace('50\n-\n64 ans','50-64 ans')
    # Remplacement CSP-
    page_content=page_content.replace('CSP\n-','CSP-')
    # Remplacement EELV
    page_content=page_content.replace('Europe Ecologie \n-\nLes Verts','Europe Ecologie Les Verts')
    # Remplacement Front National
    page_content=page_content.replace('Front National','Le Rassemblement National')
    # Noms de candidats
    page_content=page_content.replace('Jean\n-\n','Jean-')
    # Remplacement /
    page_content=page_content.replace('/ \n','/ ')
    page_content=page_content.replace('\n/','/')
    # Remplacement Total .. en minuscule
    page_content=page_content.replace('TOTAL GAUCHE','Total Gauche')
    page_content=page_content.replace('TOTAL EN MARCHE / MODEM','Total en Marche / MODEM')
    page_content=page_content.replace('TOTAL DROITE','Total Droite')
    # Suppression fin de texte
    page_content=page_content.replace('\nLa cote de confiance du Président de la République','')
    page_content=page_content.replace('\nLa cote de confiance du Premier ministre','')
    # Suppression des notes de bas de pages
    page_content=page_content.replace(' (*)','')
    page_content=page_content.replace('(*)','')
    # Suppression du dernier saut de ligne
    page_content=page_content.rstrip('\n')
    return page_content

preformatage(page_content)


"Interrogation\nFiche technique\n2\n\nMode de recueil et dates de terrain\nLa\nreprésentativité\nde\n\na\nété\nassurée\nselon\nla\nméthode\ndes\nquotas\nappliquée\naux\nvariables\nsuivantes\n:\nsexe,\nâge\net\nprofession\nde\n\naprès\nstratification\npar\nrégion\net\ncatégorie\n\n.\nEchantillon de \n1 009  \npersonnes représentatif de la population française âgée de 18 ans et plus.\nInterrogation par Internet les \n4 \net \n5 mai 2020\n.\n07/05/2020\nL'Observatoire politique \n\nMai 2020"

## Extraction de la base

In [62]:
def extract_base(fp,num_page=2)->int:
    page_content=get_page_content(num_page,fp)
    text_deb='Echantillon de \n'
    if page_content.find(text_deb)==-1:
        text_deb='Echantillon de\n'
    text_fin='\npersonnes'
    i_deb=page_content.find(text_deb)+len(text_deb)
    i_fin=page_content.find(text_fin)
    base=int(page_content[i_deb:i_fin].rstrip().replace(' ',''))
    return base
extract_base(fp)

1009

## Création du modèle de Dataframe

In [63]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])

## Extraction de page formattée

In [64]:
def extract_series(data,i,columns,nom,base,date,prv_categorie)->pd.Series:
    """Extrait une serie de data à partir de l'indice i jusqu'à la prochaine valeur non digit

    Args:
        data (list): Toutes les données dans une liste 
        i (int): Indice courant dans les données
        columns (list): Liste des noms de colonnes 
        nom (str): Nom de la personnalité
        base (int): Nombre de sondés
        date (datetime.date): Date du sondage

    Returns:
        pd.Series: La serie contenant les valeurs à stocker
        int: new index
    """
    values=[nom,base,date]
    #Catégorie
    if data[i].isupper() and '+' not in data[i] and '-' not in data[i]:
        values.append(data[i])
        i+=1
    else:
        values.append(prv_categorie)
    #Groupe
    values.append(data[i])
    # On saut la cellule qui contient le 100% et qui ne sert a rien
    i+=2
    while i<len(data) and re.match('^\d+$',data[i]):
        values.append(float(data[i]))
        i+=1
    # Vérification
    while len(values)!=len(columns):
        print(values)
        j=int(input('Où faut-il ajouter un zéro ? (1-7)'))+4
        values=values[:j]+[0.0]+values[j:]
    return pd.Series(values,index=columns),i



def extract_detailled_page(page_content,df,nom,base,date)->pd.DataFrame:
    """
    Extrait une page du pdf et l'ajoute au dataframe.
    
    Parameters
    ----------
    page_content: texte formatté de la page dot l'on souhaite extraire les données
    df: Dataframe pour stocker les données
    nom: Nom de la personnalité 
    base: Base d'étude (à compléter par regression plus tard)
    date: Date du sondage

    Returns
    -------
    Dataframe
        Dataframe df avec les nouvelles données
    """
    indice_debut=page_content.find('Ensemble')
    data=page_content[indice_debut:].split('\n')
    i=0
    cat='Ensemble'
    while i < len(data):
        if not re.match('^\d+$',data[i]):
            s,i=extract_series(data,i,df.columns,nom,base,date,cat)
            cat=s.Categorie
            df=df.append(s,ignore_index=True)
    return df


## Extraction pour une personnalité

In [65]:
def extract_detailled_perso(df,fp,page_indexes,nom,base,date)->pd.DataFrame:
    """Extrait les données sur toutes les pages pour une personnalité

    Args:
        df (pandas.Dataframe): Dataframe stockant les données
        fp (str): filepath du fichier à lire
        page_indexes (list): Liste des pages à extraire
        nom (str): Nom de la personnalité
        base (int): Nb de sondés
        date (datetime.date): Date du sondate
        inline (bool): Si True modification en ligne

    Returns:
        pd.DataFrame: Dataframe étendu des nouvelles données
    """
    for i in page_indexes:
        page_content=preformatage(get_page_content(i,fp))
        df=extract_detailled_page(page_content,df,nom,base,date)
    df=df.drop_duplicates(ignore_index=True)
    return df

        

In [99]:
df=pd.DataFrame(columns=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif'])
date='2021-10-06'
filepath='rapport/{}.pdf'.format(date)
president='Emmanuel MACRON'
page_president=[11,12]
ministe='Jean CASTEX'
page_ministre=[13,14]
base=extract_base(filepath)
df=extract_detailled_perso(df,filepath,page_president,president,base,date)
df=extract_detailled_perso(df,filepath,page_ministre,ministe,base,date)
df

['Emmanuel MACRON', 1309, '2021-10-06', 'PREFERENCE PARTISANE', 'Europe Ecologie Les Verts', 22.0, 43.0, 32.0, 3.0, 22.0, 75.0]
['Emmanuel MACRON', 1309, '2021-10-06', 'PREFERENCE PARTISANE', 'Total en Marche / MODEM', 34.0, 62.0, 3.0, 1.0, 96.0, 3.0]
['Emmanuel MACRON', 1309, '2021-10-06', 'PREFERENCE PARTISANE', 'La République en Marche !', 39.0, 60.0, 1.0, 1.0, 98.0, 1.0]
['Emmanuel MACRON', 1309, '2021-10-06', 'VOTE PRESIDENTIELLE 2017 (2ND TOUR)', 'Abstention, vote blanc ou nul', 12.0, 37.0, 43.0, 8.0, 12.0, 80.0]
['Jean CASTEX', 1309, '2021-10-06', 'PREFERENCE PARTISANE', 'Total Droite', 30.0, 34.0, 33.0, 3.0, 30.0, 67.0]
['Jean CASTEX', 1309, '2021-10-06', 'PREFERENCE PARTISANE', 'Les Républicains', 34.0, 33.0, 30.0, 3.0, 34.0, 63.0]
['Jean CASTEX', 1309, '2021-10-06', 'VOTE PRESIDENTIELLE 2017 (1ER TOUR)', 'Benoît HAMON', 19.0, 32.0, 41.0, 8.0, 19.0, 72.0]
['Jean CASTEX', 1309, '2021-10-06', 'VOTE PRESIDENTIELLE 2017 (2ND TOUR)', 'Abstention, vote blanc ou nul', 11.0, 40.0, 37.

,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif
0,Emmanuel MACRON,1309,2021-10-06,Ensemble,Ensemble,8.0,27.0,25.0,33.0,7.0,35.0,58.0
1,Emmanuel MACRON,1309,2021-10-06,SEXE DU REPONDANT,Homme,11.0,27.0,24.0,33.0,5.0,38.0,57.0
2,Emmanuel MACRON,1309,2021-10-06,SEXE DU REPONDANT,Femme,5.0,27.0,27.0,31.0,10.0,32.0,58.0
3,Emmanuel MACRON,1309,2021-10-06,AGE DU REPONDANT,18-24 ans,14.0,27.0,19.0,25.0,15.0,41.0,44.0
4,Emmanuel MACRON,1309,2021-10-06,AGE DU REPONDANT,25-34 ans,8.0,20.0,21.0,40.0,11.0,28.0,61.0
...,...,...,...,...,...,...,...,...,...,...,...,...
83,Jean CASTEX,1309,2021-10-06,VOTE PRESIDENTIELLE 2017 (1ER TOUR),Marine LE PEN,2.0,14.0,31.0,48.0,5.0,16.0,79.0
84,Jean CASTEX,1309,2021-10-06,VOTE PRESIDENTIELLE 2017 (1ER TOUR),"Abstention, vote blanc ou nul",0.0,17.0,38.0,32.0,13.0,17.0,70.0
85,Jean CASTEX,1309,2021-10-06,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Emmanuel MACRON,7.0,43.0,30.0,15.0,5.0,50.0,45.0
86,Jean CASTEX,1309,2021-10-06,VOTE PRESIDENTIELLE 2017 (2ND TOUR),Marine LE PEN,4.0,15.0,27.0,50.0,4.0,19.0,77.0


## Correction des bases

In [85]:
for c  in df.Categorie.unique():
    print('{} :[{}]'.format(c,','.join(["'"+x+"'" for x in df[df.Categorie==c].Groupe.unique()])))

Ensemble :['Ensemble']
SEXE DU REPONDANT :['Homme','Femme']
AGE DU REPONDANT :['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus']
PROFESSION DU REPONDANT :['Actifs','Artisan, commerçant','CSP+','Cadre, profession intellectuelle supérieure','Profession intermédiaire','CSP-','Employé','Ouvrier','Inactifs','Retraité','Autre Inactif']
CATEGORIE D'AGGLOMERATION :['Communes rurales','Communes urbaines de province','De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus','Agglomération parisienne']
PREFERENCE PARTISANE :['Total Gauche','La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s / PRG','Total en Marche / MODEM','La République en Marche !','Total Droite','Les Républicains','Le Rassemblement National','Sans préférence partisane']
VOTE PRESIDENTIELLE 2017 (1ER TOUR) :['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul']
VOTE PRESIDENTIELLE 

In [86]:
# CategorieX, GroupeX, CategorieY, GroupeY
relations=[
    ('SEXE DU REPONDANT' ,['Homme','Femme'],'Ensemble','Ensemble'),
    ('AGE DU REPONDANT' ,['18-24 ans','25-34 ans','35-49 ans','50-64 ans','65 ans et plus'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Actifs','Inactifs'],'Ensemble','Ensemble'),
    ('PROFESSION DU REPONDANT' ,['Artisan, commerçant','CSP+','CSP-'],'PROFESSION DU REPONDANT','Actifs'),
    ('PROFESSION DU REPONDANT' ,['Cadre, profession intellectuelle supérieure','Profession intermédiaire'],'PROFESSION DU REPONDANT','CSP+'),
    ('PROFESSION DU REPONDANT' ,['Employé','Ouvrier'],'PROFESSION DU REPONDANT','CSP-'),
    ('PROFESSION DU REPONDANT' ,['Retraité','Autre Inactif'],'PROFESSION DU REPONDANT','Inactifs'),
    ("CATEGORIE D'AGGLOMERATION" ,['Communes rurales','Communes urbaines de province','Agglomération parisienne'],'Ensemble','Ensemble'),
    ("CATEGORIE D'AGGLOMERATION" ,['De 2.000 à moins de 20.000 hab.','De 20.000 à moins de 100.000 hab.','100.000 hab. et plus'],"CATEGORIE D'AGGLOMERATION",'Communes urbaines de province'),
    ('PREFERENCE PARTISANE' ,['Total Gauche','Total en Marche / MODEM','Total Droite','Le Rassemblement National','Sans préférence partisane'],'Ensemble','Ensemble'),
    ('PREFERENCE PARTISANE' ,['La France Insoumise','Europe Ecologie Les Verts','Parti Socialiste / Génération.s / PRG'],'PREFERENCE PARTISANE','Total Gauche'),
    ('PREFERENCE PARTISANE' ,['La République en Marche !'],'PREFERENCE PARTISANE','Total en Marche / MODEM'),
    ('PREFERENCE PARTISANE' ,['Les Républicains'],'PREFERENCE PARTISANE','Total Droite'),
    ('VOTE PRESIDENTIELLE 2017 (1ER TOUR)' ,['Jean-Luc MELENCHON','Benoît HAMON','Emmanuel MACRON','François FILLON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),
    ('VOTE PRESIDENTIELLE 2017 (2ND TOUR)' ,['Emmanuel MACRON','Marine LE PEN','Abstention, vote blanc ou nul'],'Ensemble','Ensemble'),   
]

In [100]:

df=recompute_base(df,relations,df.columns[5:])
df[['Categorie','Base']].where(df.Nom==df.Nom.unique()[0]).groupby(by='Categorie').sum()

,Base
Categorie,
AGE DU REPONDANT,1309.0
CATEGORIE D'AGGLOMERATION,2252.0
Ensemble,1309.0
PREFERENCE PARTISANE,1801.0
PROFESSION DU REPONDANT,3325.0
SEXE DU REPONDANT,1309.0
VOTE PRESIDENTIELLE 2017 (1ER TOUR),1309.0
VOTE PRESIDENTIELLE 2017 (2ND TOUR),1309.0


In [101]:
# pd.to_pickle(df,'data/{}.p'.format(date))

# Resultats de toutes les personnalités
## Extraction des valeurs

In [103]:

def preformatage_personnalite(page_content)->str:
    page_content=page_content.replace('\n-\n','\n-')
    page_content=page_content.replace('\n(NP*) Personnalités non testées dans la baromètre le mois précédent','')
    page_content=page_content.replace('NP*','0')
    page_content=page_content.replace('NP','0')
    return page_content

def first_index_not_numerical(list_content)->int:
    """Retourne l'index de la première valeur qui ne soit pas une chaine de caractère représentant un nombre

    Args:
        list_content (List[str]): Liste du contenant une ligne du texte par cellule

    Returns:
        int: index de la position recherchée. Retourne -1 si non trouvée
    """
    for i in range(len(list_content)):
        if not re.match('^\d+$',list_content[i]):
            return i
    return -1
    

def extract_detailled_page_all_personnalite(page_content,columns,base,date)->pd.DataFrame:
    # Données fournies dès le début de la page
    data=page_content.split('\n')
    index_names=first_index_not_numerical(data)
    # Création des données pour faire le dataframe
    values=np.array([int(x) for x in data[:index_names]]).reshape(5,-1).T
    nb_perso=values.shape[0]

    total_pos=values[:,0:2].sum(axis=1).reshape(-1,1)
    total_neg=values[:,2:4].sum(axis=1).reshape(-1,1)

    noms=data[index_names:index_names+nb_perso]
    noms=['{} {}'.format(x.split(' ')[0],x.split(' ')[1].upper()) for x in noms]
    noms=np.array(noms).reshape(-1,1)
    

    variations=np.array(data[-1-nb_perso:-1])
    variations[variations=='=']=0
    variations=variations.reshape(-1,1)

    base_np=np.array([[base]]*nb_perso)

    date_np=np.array([[date]]*nb_perso)

    cat=np.array([['Ensemble']]*nb_perso)

    groupe=np.array([['Ensemble']]*nb_perso)

    # Création du tableau numpy
    tab=np.append(noms,base_np,axis=1)
    tab=np.append(tab,date_np,axis=1)
    tab=np.append(tab,cat,axis=1)
    tab=np.append(tab,groupe,axis=1)
    tab=np.append(tab,values,axis=1)
    tab=np.append(tab,total_pos,axis=1)
    tab=np.append(tab,total_neg,axis=1)
    tab=np.append(tab,variations,axis=1)
    df=pd.DataFrame(tab,columns=columns)
    return df

def extract_personnalite(fp,num_page,columns):
    page_content=preformatage_personnalite(get_page_content(num_page,fp))
    df=extract_detailled_page_all_personnalite(page_content,columns,base,date)
    return df



In [104]:
COLUMNS_PERSO=['Nom','Base','Date','Categorie','Groupe','tres_positif','positif','negatif','tres_negatif','nsp','total_positif','total_negatif','evolution_positif']
DTYPES_PERSO={'Nom':str,'Base':float,'Date':'datetime64[ns]','Categorie':str,'Groupe':str,'tres_positif':float,'positif':float,'negatif':float,'tres_negatif':float,'nsp':float,'total_positif':float,'total_negatif':float,'evolution_positif':float}


filepath='rapport/{}.pdf'.format(date)
base=extract_base(filepath)
df_perso=extract_personnalite(filepath,16,COLUMNS_PERSO)
df_perso=df_perso.astype(DTYPES_PERSO)
df_perso

# pd.to_pickle(df_perso,'data_perso/{}.p'.format(date))

ValueError: could not convert string to float: 'S/T'

In [94]:
df_perso

,Nom,Base,Date,Categorie,Groupe,tres_positif,positif,negatif,tres_negatif,nsp,total_positif,total_negatif,evolution_positif
0,Edouard PHILIPPE,1000,2021-09-01,Ensemble,Ensemble,14,35,16,15,20,49,31,+1
1,Nicolas HULOT,1000,2021-09-01,Ensemble,Ensemble,11,30,18,18,23,41,36,+2
2,Roselyne BACHELOT,1000,2021-09-01,Ensemble,Ensemble,6,33,20,19,22,39,39,+2
3,Nicolas SARKOZY,1000,2021-09-01,Ensemble,Ensemble,8,24,23,29,16,32,52,+1
4,Olivier VÉRAN,1000,2021-09-01,Ensemble,Ensemble,5,27,19,24,25,32,43,+1
5,Xavier BERTRAND,1000,2021-09-01,Ensemble,Ensemble,6,25,22,18,29,31,40,0
6,Bruno LE,1000,2021-09-01,Ensemble,Ensemble,5,26,18,20,31,31,38,-1
7,François BAROIN,1000,2021-09-01,Ensemble,Ensemble,5,23,18,16,38,28,34,0
8,Valérie PÉCRESSE,1000,2021-09-01,Ensemble,Ensemble,5,23,20,22,30,28,42,+3
9,Jean-Yves LE,1000,2021-09-01,Ensemble,Ensemble,5,22,16,17,40,27,33,+1


In [ ]:

# with open("rapport/2020-10-14.pdf","rb") as pdf_file:
#     pdf_report= PyPDF2.PdfFileReader(pdf_file)
#     isEncrypted = pdf_report.getIsEncrypted()
#     nbPages= pdf_report.numPages

#     page1=pdf_report.getPage(15)
#     text=page1.extractText().replace(',','.')
#     text = extract_text(pdf_file,page_numbers=[14])

# print(text)

print(df.isnull().values.any())
df.head(10)
df.describe()
df[df.Base==0]

## Find mistakes and try to correct them

In [ ]:
def find_fav_mistake(df):
    tfav=df['Très favorable']
    pfav=df['Plutôt favorable']
    stfav=df['ST Favorable']
    fav_df= df[np.abs(tfav+pfav-stfav)>0.11]
    return fav_df

def find_def_mistake(df):
    tdef=df['Très défavorable']
    pdef=df['Plutôt défavorable']
    stdef=df['ST Défavorable']
    def_df= df[np.abs(tdef+pdef-stdef)>0.11]
    return def_df

def find_nsp_mistake(df):
    stfav=df['ST Favorable']
    stdef=df['ST Défavorable']
    nsp=df['Nsp']
    tot_df= df[np.abs(stfav+stdef+nsp-100)>0.11]
    return tot_df

def find_mistake(df):
    fav_df= find_fav_mistake(df)
    def_df= find_def_mistake(df)
    tot_df=find_nsp_mistake(df)
    error_df=pd.concat([fav_df,def_df,tot_df])
    return error_df

In [ ]:
def realign_fav(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très favorable","Plutôt favorable","ST Favorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_fav(df,i+1)
    return df

def realign_def(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["Très défavorable","Plutôt défavorable","ST Défavorable"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_def(df,i+1)
    return df

def realign_nsp(df,i=0,imax=-1):
    df_err=find_fav_mistake(df)
    zeros_cols= [x for x in ["ST Favorable","ST Défavorable","Nsp"] if df[x].all()==0]
    if imax==-1:
        imax=len(zeros_cols)*len(df)
    if not df_err.empty and i<=imax:
        col=zeros_cols[0]
        values=df_err[col].tolist()
        values=values[1:]+[values[0]]
        df.loc[df_err.index,col]=values
        df=realign_nsp(df,i+1)
    return df

    



In [ ]:
def correct_mistakes(df):
    df_err=find_fav_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_fav(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_def_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_def(df_err[cond])
    df.loc[df_err.index]=df_err

    df_err=find_nsp_mistake(df)
    for nom in df_err.Nom.unique():
        for cat in df_err[df_err.Nom==nom].Categorie.unique():
            cond=((df_err.Nom==nom) & (df_err.Categorie==cat))
            df_err.loc[cond]=realign_nsp(df_err[cond])
    df.loc[df_err.index]=df_err
    return df

In [ ]:
df_err=find_mistake(df)
if len(df_err)>0:
    index=df_err.iloc[0:1].index[0]
    val=df_err.iloc[0].values[3:]
    print(val)
    correct=[]
    cols=["Base","Très favorable","Plutôt favorable","ST Favorable","Plutôt défavorable","Très défavorable","ST Défavorable","Nsp","Evolution ST Favorable","Evolution ST Défavorable"]
    while len(correct)!=10:
        correct=input()
        correct=[float(x) for x in correct.split(' ')]
    print(correct)
    df.loc[index,cols]=correct

In [ ]:
df_err=find_mistake(df)
df_err

In [ ]:

# df.to_pickle('data/'+filename+'.p')